<a href="https://colab.research.google.com/github/onkarlolge01/Machine-Learning/blob/main/Machine_Learning_Lab_onkar_lolge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <a name="0">Machine Learning Lab

Build a classfier to predict the __label__ field (substitute or not substitute) of the product substitute dataset.

### Final Project Problem: Product Substitute Prediction

__Problem Definition__:
Given a pair of products, (A, B), we say that B is a "substitute" for A if a customer would buy B in place of A -- say, if A were out of stock.

The goal of this project is to predict a substitute relationship between pairs of products. Complete the tasks in this notebook and submit your notebook via Colab  

1. <a href="#1">Read the datasets</a> (Given) 
2. <a href="#2">Data Processing</a> (Implement)
    * <a href="#21">Exploratory Data Analysis</a>
    * <a href="#22">Select features to build the model</a> (Suggested)
    * <a href="#23">Train - Validation - Test Datasets</a>
    * <a href="#24">Data Processing with Pipeline</a>
3. <a href="#3">Train (and Tune) a Classifier on the Training Dataset</a> (Implement)
4. <a href="#3">Make Predictions on the Test Dataset</a> (Implement)


__Datasets and Files:__


* __training.csv__: Training data with product pair features and corresponding labels:
> - `ID:` ID of the record
> - `label:` Tells whether the key and candidate products are substitutes (1) or not (0).
> - `key_asin ...:` Key product ASIN features 
> - `cand_asin ...:` Candidate product ASIN features 


* __public_test_features.csv__: Test data with product pairs features __without__ labels:
> - `ID:` ID of the record
> - `key_asin ...:` Key product ASIN features 
> - `cand_asin ...:` Candidate product ASIN features 


* __metadata-dataset.xlsx__: Provides detailed information about all key_ and cand_ columns in the training and test sets. Try to select some useful features to include in the model, as not all of them are suitable. `|Region Id|MarketPlace Id|ASIN|Binding Code|binding_description|brand_code|case_pack_quantity|, ...`


## 1. <a name="1">Read the datasets</a> (Given)
(<a href="#0">Go to top</a>)
</br>
<a href="https://propensity-labs-screening.s3.amazonaws.com/machine_learning/ml_data.zip">Download Dataset</a>

Then, we read the __training__ and __test__ datasets into dataframes

In [1]:
import pandas as pd
df_train=pd.read_csv('training.csv')
df_test=pd.read_csv('public_test_features.csv')

<ipython-input-1-0ea4c249788e>:2: DtypeWarning: Columns (22,26,34,41,66,79,80,88,99,129,134,139,147,154,156,157,179,194,201,212,213,214,221) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train=pd.read_csv('training.csv')
<ipython-input-1-0ea4c249788e>:3: DtypeWarning: Columns (21,25,33,40,87,107,133,138,146,153,178,191,192,200,211,220) have mixed types. Specify dtype option on import or set low_memory=False.
  df_test=pd.read_csv('public_test_features.csv')


In [2]:
df_train.head()

,ID,label,key_Region Id,key_MarketPlace Id,key_ASIN,key_Binding Code,key_binding_description,key_brand_code,key_case_pack_quantity,key_classification_code,...,cand_pkg_weight,cand_pkg_weight_uom,cand_pkg_width,cand_release_date_embargo_level,cand_dw_creation_date,cand_dw_last_updated,cand_is_deleted,cand_last_updated,cand_version,cand_external_testing_certification
0,34016,0,1,1,B00YCZ6IKA,kitchen,Kitchen,NICLW,NaN,base_product,...,0.529104,pounds,5.118110,NaN,18-Apr-13,14-Oct-17,N,13-Oct-17,2867.0,NaN
1,3581,0,1,1,B00U25WT7A,office_product,Office Product,NaN,NaN,base_product,...,0.100000,pounds,4.500000,NaN,19-May-16,21-Mar-18,N,20-Mar-18,65.0,NaN
2,36025,1,1,1,B011BZ3GXU,consumer_electronics,Electronics,NaN,NaN,base_product,...,0.654773,pounds,3.937008,NaN,10-Dec-15,16-Feb-18,N,15-Feb-18,1532.0,NaN
3,42061,1,1,1,B0089XDG3I,pc,Personal Computers,NaN,NaN,base_product,...,3.549442,pounds,10.314961,NaN,19-Oct-12,15-Feb-18,N,14-Feb-18,13964.0,NaN
4,14628,1,1,1,B014UTSBZW,miscellaneous,Misc.,ZUKC7,1.0,base_product,...,0.396832,pounds,5.196850,NaN,26-Jul-12,9-Mar-18,N,9-Mar-18,1253.0,NaN


In [7]:
df_test.head()

,ID,key_Region Id,key_MarketPlace Id,key_ASIN,key_Binding Code,key_binding_description,key_brand_code,key_case_pack_quantity,key_classification_code,key_classification_description,...,cand_pkg_weight,cand_pkg_weight_uom,cand_pkg_width,cand_release_date_embargo_level,cand_dw_creation_date,cand_dw_last_updated,cand_is_deleted,cand_last_updated,cand_version,cand_external_testing_certification
0,35057,1,1,B0096M8VR2,pc,Personal Computers,NaN,1.0,base_product,Base Product,...,0.925932,pounds,5.826772,NaN,10-Apr-13,5-Jul-16,N,4-Jul-16,699.0,NaN
1,41573,1,1,B00EAQJCWW,kitchen,Kitchen,BUNN9,2.0,base_product,Base Product,...,NaN,NaN,NaN,NaN,17-Mar-16,17-Mar-16,N,17-Mar-16,2.0,NaN
2,44029,1,1,B013P93YOQ,toy,Toy,NaN,NaN,base_product,Base Product,...,NaN,NaN,NaN,NaN,23-Dec-15,2-Dec-17,N,2-Dec-17,17.0,NaN
3,6462,1,1,B00SKJPKGW,wireless_phone_accessory,Wireless Phone Accessory,PIQ22,1.0,base_product,Base Product,...,6.250000,pounds,9.700000,NaN,22-Jan-15,18-Jan-17,N,18-Jan-17,25351.0,NaN
4,17533,1,1,B001DCEKXM,sports,Sports,SUUNR,1.0,base_product,Base Product,...,0.176368,pounds,3.228346,NaN,4-Jan-11,16-Nov-17,N,16-Nov-17,7424.0,NaN


## 2. <a name="2">Data Processing</a> (Implement)
(<a href="#0">Go to top</a>) 

### 2.1 <a name="21">Exploratory Data Analysis</a>

We look at number of rows, columns, and some simple statistics of the datasets.

In [5]:
# Implement EDA here
df_train.shape

(8282, 228)

In [60]:
df_test.shape

(8291, 227)

In [12]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8282 entries, 0 to 8281
Columns: 228 entries, ID to cand_external_testing_certification
dtypes: float64(91), int64(11), object(126)
memory usage: 14.4+ MB


In [13]:
df_train.describe()

,ID,label,key_Region Id,key_MarketPlace Id,key_case_pack_quantity,key_country_of_origin,key_discontinued_date,key_ean,key_excluded_direct_browse_node_id,key_fedas_id,...,cand_recall_notice_receive_date,cand_unit_count,cand_upc,cand_variation_theme_id,cand_video_game_region,cand_pkg_height,cand_pkg_length,cand_pkg_weight,cand_pkg_width,cand_version
count,8282.000000,8282.000000,8282.0,8282.0,3654.000000,0.0,0.0,7.565000e+03,2.254000e+03,6.0,...,0.0,1048.000000,6.113000e+03,3234.000000,4.000000,6598.000000,6598.000000,6549.000000,6597.000000,8281.000000
mean,26184.437213,0.506882,1.0,1.0,8.573344,NaN,NaN,1.728247e+12,1.975758e+09,100954.0,...,NaN,31.552672,6.164237e+11,35.931045,2.000000,3.499324,12.645653,5.709448,7.740606,11810.321217
std,15185.880921,0.499983,0.0,0.0,58.391811,NaN,NaN,2.428709e+12,3.588923e+09,0.0,...,NaN,132.072530,2.795363e+11,84.236682,1.154701,3.873965,11.608593,19.820081,6.206534,77002.176266
min,2.000000,0.000000,1.0,1.0,0.000000,NaN,NaN,1.427901e+07,1.722820e+05,100954.0,...,NaN,0.000000,3.459928e+06,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,2.000000
25%,12904.000000,0.000000,1.0,1.0,1.000000,NaN,NaN,6.473360e+11,1.064954e+06,100954.0,...,NaN,1.000000,6.122510e+11,2.000000,1.000000,1.100000,6.300000,0.249122,3.900000,54.000000
50%,26153.000000,1.000000,1.0,1.0,1.000000,NaN,NaN,7.571840e+11,1.657930e+08,100954.0,...,NaN,1.000000,7.148340e+11,8.000000,2.000000,2.400000,9.200000,0.750000,5.900000,194.000000
75%,39271.750000,1.000000,1.0,1.0,6.000000,NaN,NaN,8.822250e+11,2.611212e+09,100954.0,...,NaN,12.000000,7.939510e+11,24.000000,3.000000,4.300000,14.300000,2.780000,10.000000,1142.000000
max,52574.000000,1.000000,1.0,1.0,1700.000000,NaN,NaN,9.825480e+12,1.634073e+10,100954.0,...,NaN,3000.000000,9.999950e+11,1437.000000,3.000000,74.000000,163.000000,429.000000,97.000000,635453.000000


In [67]:
df_train['label'].value_counts()

1    4198
0    4084
Name: label, dtype: int64

In [11]:
list(df_train.columns)

['ID',
 'label',
 'key_Region Id',
 'key_MarketPlace Id',
 'key_ASIN',
 'key_Binding Code',
 'key_binding_description',
 'key_brand_code',
 'key_case_pack_quantity',
 'key_classification_code',
 'key_classification_description',
 'key_color_map',
 'key_country_of_origin',
 'key_cpsia_cautionary_statement',
 'key_creation_date',
 'key_currency_code',
 'key_customer_return_method',
 'key_customer_return_policy',
 'key_delivery_option',
 'key_discontinued_date',
 'key_ean',
 'key_esrb_age_rating',
 'key_esrb_descriptors',
 'key_excluded_direct_browse_node_id',
 'key_fedas_id',
 'key_fma_qualified_price_max',
 'key_fma_override',
 'key_Product Group Code',
 'key_Product Group Description',
 'key_has_ean',
 'key_has_online_play',
 'key_has_platform',
 'key_has_recommended_browse_nodes',
 'key_has_upc',
 'key_inner_package_type',
 'key_is_adult_product',
 'key_is_advantage',
 'key_is_certified_organic',
 'key_is_conveyable',
 'key_is_discontinued',
 'key_is_manufacture_on_demand',
 'key_is_p

In [6]:
df_train.isna().sum()

ID                                        0
label                                     0
key_Region Id                             0
key_MarketPlace Id                        0
key_ASIN                                  0
                                       ... 
cand_dw_last_updated                      1
cand_is_deleted                           1
cand_last_updated                         1
cand_version                              1
cand_external_testing_certification    8154
Length: 228, dtype: int64

### 2.2 <a name="22">Select features to build the model</a> 

For a quick start, we recommend using only a few of the numerical features for both key_ and cand_ ASINs: __item_package_quantity__, __item_height__, __item_width__, __item_length__, __item_weight__, __pkg_height__, __pkg_width__, __pkg_length__, __pkg_weight__. Feel free to explore other fields from the metadata-dataset.xlsx file.


In [20]:
# Implement here
train_x=df_train[['key_item_height',
 'key_item_length',
 'key_item_package_quantity',
 'key_item_weight',
 'key_item_width','key_pkg_height',
 'key_pkg_length',
 'key_pkg_weight',
 'key_pkg_width','cand_item_height',
 'cand_item_length',
 'cand_item_package_quantity',
 'cand_item_weight',
 'cand_item_width','cand_pkg_height',
 'cand_pkg_length',
 'cand_pkg_weight',
 'cand_pkg_width']]
train_y=df_train['label']

In [56]:
test_data=df_test[['key_item_height',
 'key_item_length',
 'key_item_package_quantity',
 'key_item_weight',
 'key_item_width','key_pkg_height',
 'key_pkg_length',
 'key_pkg_weight',
 'key_pkg_width','cand_item_height',
 'cand_item_length',
 'cand_item_package_quantity',
 'cand_item_weight',
 'cand_item_width','cand_pkg_height',
 'cand_pkg_length',
 'cand_pkg_weight',
 'cand_pkg_width']]

In [21]:
train_x.head()

,key_item_height,key_item_length,key_item_package_quantity,key_item_weight,key_item_width,key_pkg_height,key_pkg_length,key_pkg_weight,key_pkg_width,cand_item_height,cand_item_length,cand_item_package_quantity,cand_item_weight,cand_item_width,cand_pkg_height,cand_pkg_length,cand_pkg_weight,cand_pkg_width
0,1.00,86.00,1.0,6.000000,66.00,10.0,20.0,6.300000,15.0,0.00000,40.0000,1.0,0.530000,18.00000,1.574803,18.110236,0.529104,5.118110
1,2.00,2.50,6.0,NaN,0.10,0.2,4.8,0.022046,4.0,0.30000,6.7500,1.0,0.110231,4.50000,0.300000,6.750000,0.100000,4.500000
2,0.83,5.94,1.0,0.789375,2.24,2.1,7.2,1.050000,4.6,0.86614,2.3622,1.0,0.396832,3.62204,2.007874,5.236220,0.654773,3.937008
3,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2.36000,10.2400,1.0,3.480000,20.29000,2.401575,20.590551,3.549442,10.314961
4,9.33,2.75,1.0,0.438000,7.50,0.2,9.2,0.250000,7.5,8.50000,11.7500,1.0,NaN,9.87500,1.102362,7.874016,0.396832,5.196850


In [22]:
train_y

0       0
1       0
2       1
3       1
4       1
       ..
8277    1
8278    1
8279    1
8280    0
8281    1
Name: label, Length: 8282, dtype: int64

In [23]:
train_x.isna().sum()

key_item_height               2299
key_item_length               2299
key_item_package_quantity      683
key_item_weight               3172
key_item_width                2299
key_pkg_height                 775
key_pkg_length                 775
key_pkg_weight                 812
key_pkg_width                  775
cand_item_height              3395
cand_item_length              3395
cand_item_package_quantity    1174
cand_item_weight              3988
cand_item_width               3395
cand_pkg_height               1684
cand_pkg_length               1684
cand_pkg_weight               1733
cand_pkg_width                1685
dtype: int64

In [57]:
test_data.isna().sum()

key_item_height               2227
key_item_length               2227
key_item_package_quantity      644
key_item_weight               3176
key_item_width                2228
key_pkg_height                 741
key_pkg_length                 741
key_pkg_weight                 788
key_pkg_width                  741
cand_item_height              3329
cand_item_length              3329
cand_item_package_quantity    1231
cand_item_weight              3905
cand_item_width               3329
cand_pkg_height               1660
cand_pkg_length               1660
cand_pkg_weight               1722
cand_pkg_width                1660
dtype: int64

In [24]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="mean")
train_data = imputer.fit_transform(train_x)

In [58]:
testing_data=imputer.fit_transform(test_data)

In [26]:
len(train_data)

8282

In [59]:
len(testing_data)

8291

### 2.3 <a name="23">Train - Validation Datasets</a>
(<a href="#2">Go to Data Processing</a>)

We already have training and test datasets, however the test dataset is missing the labels - the goal of the project is to predict these labels.

To produce a validation set to evaluate model performance before submitting  split the training dataset into train and validation. Validation data you get here will be used later in section 3 to tune your classifier.

In [29]:
# Implement here
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train_data, train_y, test_size=0.30, random_state=42)

In [31]:
y_train

3550    1
5927    0
7287    0
2328    0
29      1
       ..
5734    0
5191    0
5390    0
860     0
7270    0
Name: label, Length: 5797, dtype: int64

### 2.4 <a name="24">Data processing with Pipeline</a>

Build a [pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html)  to impute the missing values and scale the numerical features, and finally train the classifier on the imputed and scaled dataset. 


In [34]:
# Implement here
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
pipe = Pipeline([('scaler', StandardScaler()), ('RandomForestClassifier', RandomForestClassifier())])
pipe.fit(X_train, y_train)


Pipeline(steps=[('scaler', StandardScaler()),
                ('RandomForestClassifier', RandomForestClassifier())])

In [37]:
Pipeline(steps=[('scaler', StandardScaler()), ('RandomForestClassifier', RandomForestClassifier())])
print('Training score-',pipe.score(X_train, y_train))
print('Validation score-',pipe.score(X_val, y_val))

Training score- 0.9799896498188718
Validation score- 0.6016096579476862


## 3. <a name="3">Train (and Tune) a Classifier</a> (Implement)
(<a href="#0">Go to top</a>)

Train and tune the classifier

In [45]:
# Implement here
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 1000, num = 3)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 100, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [200, 600, 1000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 19, 28, 37, 46, 55, 64, 73, 82, 91, 100, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [46]:
# base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 50, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_val, y_val)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=50,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 19, 28, 37, 46, 55,
                                                      64, 73, 82, 91, 100,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 600, 1000]},
                   random_state=42, verbose=2)

In [47]:
rf_random.best_params_

{'n_estimators': 600,
 'min_samples_split': 5,
 'min_samples_leaf': 4,
 'max_features': 'auto',
 'max_depth': 19,
 'bootstrap': True}

In [52]:
model=RandomForestClassifier(n_estimators=600,
 min_samples_split= 5,
 min_samples_leaf= 4,
 max_features='auto',
 max_depth=19,
 bootstrap= True)
model.fit(X_train, y_train)

/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


RandomForestClassifier(max_depth=19, max_features='auto', min_samples_leaf=4,
                       min_samples_split=5, n_estimators=600)

In [54]:
from sklearn.metrics import accuracy_score
train_accuracy=accuracy_score(y_train,model.predict(X_train))
val_accuracy=accuracy_score(y_val,model.predict(X_val))

In [55]:
print('Train accuracy-',train_accuracy)
print('Validation accuracy-',val_accuracy)

Train accuracy- 0.9341038468173193
Validation accuracy- 0.606841046277666


## 4. <a name="4">Make Predictions on the Test Dataset</a> (Implement)
(<a href="#0">Go to top</a>)

Use the trained classifier to predict the labels on the test set. Test accuracy would be displayed upon a valid submission to the leaderboard.

In [64]:
# Implement here

# Get test data to test the classifier
# ! test data should come from public_test_features.csv !
# ...
test_pred=model.predict(testing_data)
# Use the trained model to make predictions on the test dataset
# test_predictions = ...


In [65]:
test_pred

array([1, 0, 0, ..., 1, 0, 0])

In [66]:
len(test_pred)

8291